In [ ]:
!pip install pypdf2
!pip install requests
!pip install pycryptodom

In [ ]:
!pip install beautifulsoup4
!pip install joblib

In [ ]:
import os
import pathlib

# カレントディレクトリを取得
current_dir = pathlib.Path(os.getcwd())
# カレントディレクトリの親ディレクトリを取得して ROOT_PATH とする
ROOT_PATH = current_dir.parent
# OUTPUT_PATH を設定
OUTPUT_PATH = os.path.join(ROOT_PATH, "output")

# ダウンロードするPDFファイルを保存するディレクトリのパス
download_dir = os.path.join(OUTPUT_PATH, 'pdf_downloads')
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

In [ ]:
import requests
import os
from bs4 import BeautifulSoup
from joblib import Parallel, delayed

# クロールするウェブサイトのURL
urls = ["https://data.e-gov.go.jp/data/dataset/?q=報告書&res_format=PDF&res_license_id=cc-by&sort=score+desc%2C+metadata_modified+desc-20",
        "https://data.e-gov.go.jp/data/dataset/?q=報告書&res_format=PDF&res_license_id=gjstu&sort=score+desc%2C+metadata_modified+desc-20",
        "https://data.e-gov.go.jp/data/dataset/?q=報告書&res_format=PDF&res_license_id=CC-BY&sort=score+desc%2C+metadata_modified+desc-20",
        "https://data.e-gov.go.jp/data/dataset/?q=白書&res_format=PDF&res_license_id=cc-by&sort=score+desc%2C+metadata_modified+desc-20",
        "https://data.e-gov.go.jp/data/dataset/?q=白書&res_format=PDF&res_license_id=gjstu&sort=score+desc%2C+metadata_modified+desc-20",
        "https://data.e-gov.go.jp/data/dataset/?q=白書&res_format=PDF&res_license_id=CC-BY&sort=score+desc%2C+metadata_modified+desc-20",]

# クロールするぺージ数
last_pages = [211, 40, 4, 108, 10, 9]

def crawl_pages(url, last_page=384, output_dir="pdf_downloads"):
    crawl_search_pdf(url, output_dir=output_dir)
    print("page1をクロールしました。")
    for next in range(2, last_page+1):
        next_url = url + "&page=" + str(next)
        crawl_search_pdf(next_url, output_dir=output_dir)
        print("page" + str(next) + "をクロールしました。")

def crawl_search_pdf(url, output_dir="pdf_downloads"):
    """指定されたURLからPDFリンクを見つける"""
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    li_tag = soup.find_all('li', class_='list_dataset_item')
    
    for target in li_tag:
        for link in target.find_all('a', string=['PDF', 'pdf'], href=True):
            href = link.get('href')
            crawl_all_pdfs("https://data.e-gov.go.jp"+href, output_dir=output_dir)
    
def crawl_all_pdfs(url, output_dir="pdf_downloads"):
    """指定されたURLからPDFリンクを見つけてダウンロードする"""
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    li_tag = soup.find_all('li', class_='list_dataset_item resource-item')

    # PDFリンクのリストをリスト内包表記で抽出
    pdf_links = [[link.get('href') for link in target.find_all('a') if link.get('href').endswith('.pdf')] for target in li_tag]
    filtered_pdf_links = [element for element in pdf_links if element]
    Parallel(n_jobs=1, verbose=0)([delayed(write_pdf_links_to_file)(pdf_link[0], output_dir) for pdf_link in filtered_pdf_links])

def write_pdf_links_to_file(pdf_link, output_dir="pdf_downloads"):
    download_file = os.path.join(output_dir, 'urls.txt') 
    # PDFリンクをファイルに書き込む
    with open(download_file, "a", encoding="utf-8") as file:
        file.write(pdf_link + "\n")  # "\n" は新しい行を開始します

# クローリングを開始する
for url, last_page in zip(urls,last_pages):
    crawl_pages(url, last_page=last_page, output_dir=download_dir)

In [ ]:
import requests
import PyPDF2
import re
import json
import os
from io import BytesIO

def extract_text_from_pdf_with_pypdf2(url):
    # PDFファイルをダウンロードする
    response = requests.get(url)
    response.raise_for_status()  # HTTPエラーをチェック
    texts = []
    json_text = {}
    # ダウンロードしたPDFファイルをメモリ上で開く
    with BytesIO(response.content) as f:
        # PyPDF2を使用してPDFファイルを読み込む
        pdf_reader = PyPDF2.PdfReader(f)
        text = ""
        
        # 各ページからテキストを抽出
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text() if page.extract_text() else ''
        
    without_kaigyou_text = text.replace('\n', '')
    cleaned_text = re.sub(r'\.\.+', '', without_kaigyou_text)

    json_text['text'] = cleaned_text 
    texts.append(json_text)
    return texts

# 使用例
# if not os.path.exists(download_dir):
#     os.makedirs(download_dir)

# url = "https://www.ppc.go.jp/files/pdf/nichibeiou_ekkyouiten_report.pdf"
# texts = extract_text_from_pdf_with_pypdf2(url)
# # JSONLファイルに保存
# with open(os.path.join(download_dir, 'scraped_data.jsonl'), 'a', encoding='utf-8') as file:
#     for data in texts:
#         json_line = json.dumps(data, ensure_ascii=False)  # JSON形式の文字列に変換
#         file.write(json_line + '\n')  
# print(texts[0])  

# urls.txtに保存されたURLからPDFをダウンロードしてテキストを抽出
with open(os.path.join(download_dir, 'urls.txt'), 'r', encoding='utf-8') as file:
    urls = file.readlines()
    for url in urls:
        try:
            texts = extract_text_from_pdf_with_pypdf2(url.replace('\n', ''))
            # JSONLファイルに保存
            with open(os.path.join(download_dir, 'scraped_data.jsonl'), 'a', encoding='utf-8') as file:
                for data in texts:
                    json_line = json.dumps(data, ensure_ascii=False)  # JSON形式の文字列に変換
                    file.write(json_line + '\n')
        except Exception as e:
            print(str(e.args) +  url)
            continue

In [ ]:
# def download_pdf(link):
#     """PDFファイルをダウンロードする関数"""
#     response = requests.get(link)
#     filename = link.split('/')[-1]
#     with open(os.path.join(OUTPUT_PATH, filename), 'wb') as f:
#         f.write(response.content)
#     print(f"{filename} をダウンロードしました。")